### If you dont want to use Wandb, disable Wandb otherwise optional

references for WANDB
https://analyticsindiamag.com/hands-on-guide-to-weights-and-biases-wandb-with-python-implementation/

https://docs.wandb.ai/


In [1]:
!nvidia-smi

Fri Nov 21 11:51:32 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   53C    P0              69W / 300W |  12746MiB / 81920MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os
print(os.getcwd())
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/workspace/bert_punct_model_final/data/Notebooks


In [3]:
!huggingface-cli login --token {hf_token}

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `llm_finetuning` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `llm_finetuning`


In [4]:
import os
os.environ["WANDB_DISABLED"]="true"

# Install required packages

In [7]:
# ! pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece]

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version

In [5]:
import transformers
print(transformers.__version__)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.53.2


# Fine-tuning a model on a translation task

In [6]:
model_checkpoint = "ai4bharat/indictrans2-en-indic-dist-200M"

### Loading the dataset

We will use the [datasets](https://github.com/huggingface/datasets/tree/master/datasets/wmt16) library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions load_dataset and load_metric. We use the English/Romanian part of the WMT dataset here

In [7]:
# !pip install datasets evaluate

In [8]:
from datasets import load_dataset
from evaluate import load
raw_datasets = load_dataset("thenlpresearcher/iitb_marathi_punct_variants")
metric = load("sacrebleu")

In [9]:
metric

EvaluationModule(name: "sacrebleu", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'e

In [10]:
from evaluate import load

# Example 1: perfect match
predictions_1 = ["hello there general kenobi", "foo bar foobar"]
references_1 = [
    ["hello there general kenobi", "hello there !"],
    ["foo bar foobar", "foo bar foobar"]
]

metric = load("sacrebleu")
results_1 = metric.compute(predictions=predictions_1, references=references_1)

print("Example 1 Results:")
print("Keys:", list(results_1.keys()))
print("BLEU Score:", round(results_1["score"], 1))
print()

# Example 2: partial match
predictions_2 = [
    "hello there general kenobi",
    "on our way to ankh morpork"
]
references_2 = [
    ["hello there general kenobi", "hello there !"],
    ["goodbye ankh morpork", "ankh morpork"]
]

results_2 = metric.compute(predictions=predictions_2, references=references_2)

print("Example 2 Results:")
print("Keys:", list(results_2.keys()))
print("BLEU Score:", round(results_2["score"], 1))

Example 1 Results:
Keys: ['score', 'counts', 'totals', 'precisions', 'bp', 'sys_len', 'ref_len']
BLEU Score: 100.0

Example 2 Results:
Keys: ['score', 'counts', 'totals', 'precisions', 'bp', 'sys_len', 'ref_len']
BLEU Score: 39.8


In [11]:
# !pip install -U "pyarrow<17" "datasets>=2.14,<3"

In [12]:
import pyarrow as pa, datasets
print("pyarrow:", pa.__version__)
print("datasets:", datasets.__version__)

pyarrow: 16.1.0
datasets: 2.21.0


The dataset object itself is [datasetdict](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasetdict), which contains one key for the training, validation and test set:

In [13]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['src_lang', 'tgt_lang', 'src', 'tgt'],
        num_rows: 379480
    })
    validation: Dataset({
        features: ['src_lang', 'tgt_lang', 'src', 'tgt'],
        num_rows: 47435
    })
    test: Dataset({
        features: ['src_lang', 'tgt_lang', 'src', 'tgt'],
        num_rows: 47435
    })
})

In [14]:
raw_datasets["train"][0]

{'src_lang': 'eng_Latn',
 'tgt_lang': 'mar_Deva',
 'src': 'Could you use a less expensive alternative or a generic or house brand item',
 'tgt': 'तुम्ही कमी खर्चाचा पर्याय किंवा सामान्य किंवा घरगुती छापाचा नग वापरू शकता का?'}

To get a sense of how the data looks like, the following function will show some examples picked randomly in the dataset.

In [15]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(raw_datasets["train"])

,src_lang,tgt_lang,src,tgt
0,eng_Latn,mar_Deva,"However, the next three seasons after that, the Bulls won the championship.","मात्र, त्यानंतरच्या तीन हंगामांमध्ये बुल्सने विजेतेपद पटकावले."
1,eng_Latn,mar_Deva,Some of the factors are believed to come from genetics.,काही घटक अनुवांशिकतेतून येतात असे मानले जाते.
2,eng_Latn,mar_Deva,She reveals that her family is moving abroad to escape the scandal and that she may never see him again,तिने उघड केले की तिचे कुटुंब या घोटाळ्यापासून वाचण्यासाठी परदेशात जात आहे आणि ती त्याला पुन्हा कधीही पाहू शकणार नाही.
3,eng_Latn,mar_Deva,It summarizes the Spoken-Tutorial project.,ज्यामध्ये तुम्हाला प्रॉजेक्टचा सारांश मिळेल.
4,eng_Latn,mar_Deva,Examples provided in this article will use TestNG syntax and annotations.,या लेखात प्रदान केलेली उदाहरणे टेस्टएनजी सिंटॅक्स आणि भाष्ये वापरतील.


You can call its compute method with your predictions and labels, which need to be list of decoded strings (list of list for the labels):

In [16]:
from evaluate import load

# Load metric
metric = load("sacrebleu")

preds = ["hello there general kenobi", "you are a bold one"]
refs  = [["hello there general kenobi"], ["you are a bold one"]]
result = metric.compute(predictions=preds, references=refs)
print("BLEU =", result["score"])

BLEU = 100.00000000000004


# Preprocessing the data

Before we can feed those texts to our model, we need to preprocess them. This is done by a Transformers Tokenizer which will (as the name indicates) tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

To do all of this, we instantiate our tokenizer with the AutoTokenizer.from_pretrained method, which will ensure:

we get a tokenizer that corresponds to the model architecture we want to use,
we download the vocabulary used when pretraining this specific checkpoint.
That vocabulary will be cached, so it's not downloaded again the next time we run the cell.

If you downloaded the model manually, you can provide model present directory instead of model_checkpoint

In [17]:
def initialize_model_and_tokenizer(ckpt_dir, quantization=None):
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, BitsAndBytesConfig
    import torch

    # Quantization setup
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)

    # Load model
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    # Move to device and optionally convert to half precision
    if qconfig is None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    # Make sure model is in training mode for fine-tuning
    model.train()

    return tokenizer, model

In [19]:
import torch
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-dist-200M"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, quantization)

A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M:
- tokenization_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M:
- configuration_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/ai4bharat/indictrans2-en-indic-dist-200M:
- modeling_indictrans.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


You can directly call this tokenizer on one sentence or a pair of sentences:

In [20]:
from IndicTransToolkit.processor import IndicProcessor

ip = IndicProcessor(inference=True)
sentences = ["Hello, this is one sentence!", "This is another sentence."]

# Preprocess sentences
preprocessed = ip.preprocess_batch(sentences, src_lang="eng_Latn", tgt_lang="mar_Deva")
print("Preprocessed sentences:", preprocessed)

Preprocessed sentences: ['eng_Latn mar_Deva Hello , this is one sentence !', 'eng_Latn mar_Deva This is another sentence .']


In [21]:
# Tokenize the batch and generate input encodings
inputs = en_indic_tokenizer(
    preprocessed,
    truncation=True,
    padding="longest",
    return_tensors="pt",
    return_attention_mask=True,
).to(DEVICE)
print(inputs)

{'input_ids': tensor([[   4,   29, 7951,    7,   36,   13,   75, 4534,   74,    2],
        [   1,    1,    4,   29,   67,   13,  309, 4534,    5,    2]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


To prepare the targets for our model, we need to tokenize them inside the as_target_tokenizer context manager. This will make sure the tokenizer uses the special tokens corresponding to the targets:

In [22]:
sentences = ["Hello, this is one sentence!", "This is another sentence."]

# Preprocess sentences
preprocessed = ip.preprocess_batch(sentences, src_lang="eng_Latn", tgt_lang="mar_Deva")
print("Preprocessed sentences:", preprocessed)

with en_indic_tokenizer.as_target_tokenizer():
    print(en_indic_tokenizer(preprocessed,
    truncation=True,
    padding="longest",
    return_tensors="pt",
    return_attention_mask=True
))

Preprocessed sentences: ['eng_Latn mar_Deva Hello , this is one sentence !', 'eng_Latn mar_Deva This is another sentence .']
{'input_ids': tensor([[67156, 59836,  3233, 11007,  7130, 60726, 59836,  2134,  5172, 43144,
         36485, 64643,     5, 45158, 14444, 46728, 78577,    40,     2],
        [67156, 59836,  3233, 11007,  7130, 60726, 59836,  2134,  5172, 43144,
         56649, 14444, 79991, 78577,     4,     2,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]])}


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [23]:
from typing import List, Dict
import torch

def decode_tokenized_batch(tokenizer, batch: Dict[str, torch.Tensor], ip=None, target_lang=None):
    """
    Decode a batch of token IDs (input_ids or labels) into readable text.
    
    Args:
        tokenizer: HuggingFace tokenizer
        batch: dictionary containing 'input_ids' or 'labels' tensors
        ip: IndicProcessor instance (optional, for postprocessing)
        target_lang: target language code, required if ip is used
    
    Returns:
        List of decoded strings
    """
    # Choose which field to decode
    if "labels" in batch:
        tokens_to_decode = batch["labels"]
    elif "input_ids" in batch:
        tokens_to_decode = batch["input_ids"]
    else:
        raise ValueError("Batch must contain 'labels' or 'input_ids'.")

    # Convert to list of lists (batch of sequences)
    if isinstance(tokens_to_decode, torch.Tensor):
        tokens_to_decode = tokens_to_decode.tolist()

    # Decode using tokenizer
    decoded_texts = tokenizer.batch_decode(
        tokens_to_decode,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    )
    
    print(decoded_texts)
    
    # Postprocess with IndicProcessor if provided
    if ip is not None and target_lang is not None:
        decoded_texts = ip.postprocess_batch(decoded_texts, lang=target_lang)

    return decoded_texts

In [24]:
raw_datasets['train'][:2]

{'src_lang': ['eng_Latn', 'eng_Latn'],
 'tgt_lang': ['mar_Deva', 'mar_Deva'],
 'src': ['Could you use a less expensive alternative or a generic or house brand item',
  'If you do not round this number, you will get a different final result than in the example.'],
 'tgt': ['तुम्ही कमी खर्चाचा पर्याय किंवा सामान्य किंवा घरगुती छापाचा नग वापरू शकता का?',
  'जर तुम्ही हा आकडा पूर्णांक केला नाही तर तुम्हाला उदाहरणापेक्षा एखादा वेगळा अंतिम निकाल मिळेल.']}

In [25]:
# ---- Define preprocessing ----
def preprocess_function(examples, tokenizer, ip):
    # Step 1: Preprocess both source and target
    src_texts = ip.preprocess_batch(
        examples["src"], src_lang="eng_Latn", tgt_lang="mar_Deva"
    )
    tgt_texts = ip.preprocess_batch(
        examples["tgt"], src_lang="mar_Deva", tgt_lang="eng_Latn"
    )

    # Step 2: Tokenize (new Transformers ≥v4.30 style)
    model_inputs = tokenizer(
        src_texts,
        text_target=tgt_texts,
        padding="longest",
        truncation=True,
        return_tensors=None,        # return lists for Dataset.map
    )

    return model_inputs

In [26]:
example_batch = preprocess_function(raw_datasets['train'][:2], en_indic_tokenizer, ip)
print(decode_tokenized_batch(en_indic_tokenizer, example_batch))

['mar_Deva eng_Latn तुम्ही कमी खर्चाचा पर्याय किंवा सामान्य किंवा घरगुती छापाचा नग वापरू शकता का?', 'mar_Deva eng_Latn जर तुम्ही हा आकडा पूर्णांक केला नाही तर तुम्हाला उदाहरणापेक्षा एखादा वेगळा अंतिम निकाल मिळेल.']
['mar_Deva eng_Latn तुम्ही कमी खर्चाचा पर्याय किंवा सामान्य किंवा घरगुती छापाचा नग वापरू शकता का?', 'mar_Deva eng_Latn जर तुम्ही हा आकडा पूर्णांक केला नाही तर तुम्हाला उदाहरणापेक्षा एखादा वेगळा अंतिम निकाल मिळेल.']


To apply this function on all the pairs of sentences in our dataset, we just use the map method of our dataset object we created earlier. This will apply the function on all the elements of all the splits in dataset, so our training, validation and testing data will be preprocessed in one single command

In [27]:
from datasets import DatasetDict

batch_size = 32
tokenized_datasets_dict = {}

# Iterate over splits
for split in raw_datasets.keys():
    print(f"Processing split: {split}")

    # Tokenize this split using your preprocessing function
    tokenized_split = raw_datasets[split].map(
        lambda x: preprocess_function(x, tokenizer=en_indic_tokenizer, ip=ip),
        batched=True,
        batch_size=batch_size,
        remove_columns=raw_datasets[split].column_names  # optional
    )

    tokenized_datasets_dict[split] = tokenized_split

# Rebuild DatasetDict
tokenized_datasets = DatasetDict(tokenized_datasets_dict)

# Optional: inspect
print(tokenized_datasets)
print(tokenized_datasets["train"][0].keys())

Processing split: train


Parameter 'function'=<function <lambda> at 0x7f83d829db40> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map: 100%|█████████████████| 379480/379480 [04:31<00:00, 1397.30 examples/s]


Processing split: validation


Map: 100%|███████████████████| 47435/47435 [00:33<00:00, 1410.04 examples/s]


Processing split: test


Map: 100%|███████████████████| 47435/47435 [00:34<00:00, 1381.62 examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 379480
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 47435
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 47435
    })
})
dict_keys(['input_ids', 'attention_mask', 'labels'])


In [28]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 379480
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 47435
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 47435
    })
})


In [29]:
tokenized_datasets.push_to_hub(
    "thenlpresearcher/iitb_en_indic_marathi_punct_variants_tokenized", 
    private=True
)

Creating parquet from Arrow format:  23%|▏| 89/380 [00:00<00:01, 175.80ba/s]
Creating parquet from Arrow format:  28%|▎| 107/380 [00:00<00:01, 170.62ba/s
Creating parquet from Arrow format:  33%|▎| 125/380 [00:00<00:01, 167.54ba/s
Creating parquet from Arrow format:  37%|▎| 142/380 [00:00<00:01, 165.70ba/s
Creating parquet from Arrow format:  42%|▍| 159/380 [00:00<00:01, 164.12ba/s
Creating parquet from Arrow format:  46%|▍| 176/380 [00:01<00:01, 163.24ba/s
Creating parquet from Arrow format:  51%|▌| 193/380 [00:01<00:01, 163.16ba/s
Creating parquet from Arrow format:  55%|▌| 210/380 [00:01<00:01, 159.48ba/s
Creating parquet from Arrow format:  60%|▌| 229/380 [00:01<00:00, 166.21ba/s
Creating parquet from Arrow format:  65%|▋| 248/380 [00:01<00:00, 171.74ba/s
Creating parquet from Arrow format:  70%|▋| 267/380 [00:01<00:00, 174.43ba/s
Creating parquet from Arrow format:  75%|▊| 286/380 [00:01<00:00, 176.60ba/s
Creating parquet from Arrow format:  81%|▊| 306/380 [00:01<00:00, 181.43ba/s

CommitInfo(commit_url='https://huggingface.co/datasets/thenlpresearcher/iitb_en_indic_marathi_punct_variants_tokenized/commit/c6dd175a36f88435dc55e4fd8bd54d1cf9368524', commit_message='Upload dataset', commit_description='', oid='c6dd175a36f88435dc55e4fd8bd54d1cf9368524', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/thenlpresearcher/iitb_en_indic_marathi_punct_variants_tokenized', endpoint='https://huggingface.co', repo_type='dataset', repo_id='thenlpresearcher/iitb_en_indic_marathi_punct_variants_tokenized'), pr_revision=None, pr_num=None)

In [30]:
from datasets import load_dataset
tokenized_datasets = load_dataset("thenlpresearcher/iitb_en_indic_marathi_punct_variants_tokenized")

Generating train split: 100%|█| 379480/379480 [00:00<00:00, 486342.03 exampl
Generating validation split: 100%|█| 47435/47435 [00:00<00:00, 542059.05 exa
Generating test split: 100%|█| 47435/47435 [00:00<00:00, 536073.77 examples/


In [31]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 379480
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 47435
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 47435
    })
})

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the AutoModelForSeq2SeqLM class. Like with the tokenizer, the from_pretrained method will download and cache the model for us.

In [32]:
print(DEVICE)

cuda


To instantiate a Seq2SeqTrainer, we will need to define three more things. The most important is the [Seq2SeqTrainingArguments](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Seq2SeqTrainingArguments), which is a class that contains all the attributes to customize the training. It requires one folder name, which will be used to save the checkpoints of the model, and all other arguments are optional:

In [62]:
from transformers import Seq2SeqTrainingArguments

model_checkpoint = en_indic_ckpt_dir
batch_size = 16
model_name = model_checkpoint.split("/")[-1]

source_lang = 'eng_Ltn'
target_lang = 'mar_Deva'

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-iitb-finetuned-{source_lang}-to-{target_lang}",
    eval_strategy="steps",
    save_strategy="steps",
    eval_steps=50,                 # evaluate every 2000 training steps
    save_steps=50,                 # save checkpoint every 2000 steps
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=False,                     # if GPU supports
    logging_steps=100,
    seed=42,
    include_inputs_for_metrics=True,
    
    # --- Add these for pushing to Hub ---
    push_to_hub=True,                                 # enables upload
    hub_model_id=f"thenlpresearcher/iitb_punct_robust_finetuned_{source_lang}_to_{target_lang}",  # your repo name
    hub_private_repo=False,                            # makes it private
    hub_strategy="end",                        # uploads at each save
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using `include_inputs_for_metrics` is deprecated and will be removed in version 5 of 🤗 Transformers. Please use `include_for_metrics` list argument instead.


In [28]:
# batch_size = 16
# model_name = model_checkpoint.split("/")[-1]
# args = Seq2SeqTrainingArguments(
#     f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
#     eval_strategy = "epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     weight_decay=0.01,
#     save_total_limit=3,
#     num_train_epochs=1,
#     predict_with_generate=True
# )

Here we set the evaluation to be done at the end of each epoch, tweak the learning rate, use the batch_size defined at the top of the cell and customize the weight decay. Since the Seq2SeqTrainer will save the model regularly and our dataset is quite large, we tell it to make three saves maximum. Lastly, we use the predict_with_generate option (to properly generate summaries) and activate mixed precision training (to go a bit faster).

Model will save under **{model_name}-finetuned-{source_lang}-to-{target_lang}** directory

Then, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels:

In [34]:
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)

In [35]:
# data_collator = DataCollatorForSeq2Seq(en_indic_tokenizer, model=en_indic_model,
#     padding=True,          # default: pad dynamically per batch
#     return_tensors="pt"
# )

The last thing to define for our Seq2SeqTrainer is how to compute the metrics from the predictions. We need to define a function for this, which will just use the metric we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [36]:
def postprocess_indictrans_text(preds, labels, source_lang="eng_Latn", target_lang="mar_Deva"):
    """
    Remove language prefix from predictions and labels for evaluation.
    For decoder outputs, the prefix is TARGET_LANG SOURCE_LANG.
    """
    prefix = f"{target_lang} {source_lang}"

    def remove_prefix(text):
        if text.startswith(prefix):
            return text[len(prefix):].strip()
        return text.strip()

    # Strip prefixes
    preds = [remove_prefix(p) for p in preds]
    labels = [[remove_prefix(l)] for l in labels]  # BLEU expects list of lists
    return preds, labels

In [32]:
# import numpy as np

# def compute_metrics_indictrans(eval_preds, tokenizer, metric, source_lang="eng_Latn", target_lang="mar_Deva"):
#     """
#     Decode model predictions and labels, remove prefixes, compute BLEU.
#     """
#     preds, labels = eval_preds
#     if isinstance(preds, tuple):
#         preds = preds[0]

#     # Decode predictions
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True, clean_up_tokenization_spaces=True)

#     # Replace -100 with pad_token_id for labels and decode
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=True)

#     # Remove prefixes or do custom postprocessing
#     decoded_preds, decoded_labels = postprocess_indictrans_text(
#         decoded_preds, decoded_labels, source_lang=source_lang, target_lang=target_lang
#     )

#     # BLEU expects references as list of lists
#     decoded_labels = [[lbl] for lbl in decoded_labels]
# #     print(decoded_labels)

#     # Compute BLEU
#     result = metric.compute(predictions=decoded_preds, references=decoded_labels)
#     result = {"bleu": result["score"]}

#     # Average generated length (non-padding tokens)
#     prediction_lens = [np.count_nonzero(p != tokenizer.pad_token_id) for p in preds]
#     result["gen_len"] = float(np.mean(prediction_lens))

#     # Round metrics for readability
#     return {k: round(v, 4) for k, v in result.items()}

In [37]:
import torch
import numpy as np
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_metric

# -----------------------------
# 1️⃣ Load model & tokenizer
# -----------------------------
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-dist-200M"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def initialize_model_and_tokenizer(ckpt_dir, quantization=None):
    tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir, trust_remote_code=True
    )
    if quantization == "fp16":
        model = model.half()
    model.to(DEVICE)
    return tokenizer, model

en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir)

# -----------------------------
# 2️⃣ Prepare small validation batch
# -----------------------------
# tokenized_datasets must already exist (from previous preprocessing)
small_val_dataset = tokenized_datasets["validation"].select(range(10))

input_ids = torch.tensor(small_val_dataset["input_ids"]).to(DEVICE)
attention_mask = torch.tensor(small_val_dataset["attention_mask"]).to(DEVICE)
labels = torch.tensor(small_val_dataset["labels"]).to(DEVICE)

# Decode reference sentences for metric
reference_sentences = [
    en_indic_tokenizer.decode(
        np.where(l.cpu().numpy() != -100, l.cpu().numpy(), en_indic_tokenizer.pad_token_id),
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    )
    for l in labels
]

# -----------------------------
# 3️⃣ Generate translations
# -----------------------------
with torch.no_grad():
    generated_tokens = en_indic_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=128,
        num_beams=5,
        early_stopping=True,
        decoder_start_token_id=en_indic_model.config.decoder_start_token_id
    )

# -----------------------------
# 4️⃣ Define postprocessing
# -----------------------------
def postprocess_indictrans_text(preds, labels, source_lang="eng_Latn", target_lang="mar_Deva"):
    """
    Remove language prefix from predictions and labels for evaluation.
    BLEU expects list of references per prediction.
    """
    prefix = f"{target_lang} {source_lang}"

    def remove_prefix(text):
        if text.startswith(prefix):
            return text[len(prefix):].strip()
        return text.strip()

    preds = [remove_prefix(p) for p in preds]
    labels = [[remove_prefix(l)] for l in labels]  # BLEU expects list of lists
    return preds, labels

# -----------------------------
# 5️⃣ Compute metrics
# -----------------------------
metric = load_metric("sacrebleu")

def compute_metrics_indictrans(eval_preds, tokenizer, metric, source_lang="eng_Latn", target_lang="mar_Deva"):
    preds, labels = eval_preds

    # Convert to numpy if tensors
    if isinstance(preds, torch.Tensor):
        preds = preds.cpu().numpy()
    if isinstance(labels, torch.Tensor):
        labels = labels.cpu().numpy()

    # Decode predictions
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    # Replace -100 with pad_token_id for labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    # Optional postprocessing
    decoded_preds, decoded_labels = postprocess_indictrans_text(decoded_preds, decoded_labels, source_lang, target_lang)

    # Compute BLEU
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    # Average generated length (non-padding tokens)
    prediction_lens = [np.count_nonzero(p != tokenizer.pad_token_id) for p in preds]
    result["gen_len"] = float(np.mean(prediction_lens))

    # Round metrics for readability
    return {k: round(v, 4) for k, v in result.items()}

# -----------------------------
# 6️⃣ Evaluate on sample batch
# -----------------------------
results = compute_metrics_indictrans(
    (generated_tokens, labels),
    tokenizer=en_indic_tokenizer,
    metric=metric
)

print("Reference sentences:", reference_sentences)
print("Generated translations:", en_indic_tokenizer.batch_decode(generated_tokens, skip_special_tokens=True))
print("Results:", results)

/tmp/ipykernel_4216/3091177656.py:79: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


Reference sentences: ['mar_Deva eng_Latn आता सुरुवात करू. मी आधीच माझ्या सिस्टिमवर Koha इन्स्टॉल केले आहे.', 'mar_Deva eng_Latn फक्त दोन खरोखरच्या दुहेरी कॉन्सर्टोपैकी एक तो आहे, जे दोन्ही मोत्झार्तने लिहिले.', 'mar_Deva eng_Latn थोडक्यात,', 'mar_Deva eng_Latn ह्या पुस्तकात आपण ह्या संकल्पनांचा वापर केला नाही पाहिजे आणि ज्यांना पुढे वाचण्याची इच्छा आहे, त्यांच्या उपयोगासाठी फक्त हा उल्लेख केला पाहिजे.', 'mar_Deva eng_Latn ती युक्रेनियन नॅशनल फिलहारमोनिकची एकटी वादक होती.', 'mar_Deva eng_Latn तो Alex किंवा phpacademy कडून आला असे म्हणू शकतो.', 'mar_Deva eng_Latn 2015 मध्ये त्याने वॉर्सा येथे दुसरे नाव्ह्याचे दुकान आणि सोपोटमध्ये एक बार उघडला.', 'mar_Deva eng_Latn उन्हाळी परीक्षांसाठी पूर्वविमोचन साहित्य दरवर्षी 31 मार्चपासून उपलब्ध असते ( आणि ते जानेवारीच्या परीक्षांसाठी 1 नोव्हेंबरला देखील उपलब्ध असेल ).', 'mar_Deva eng_Latn ह्यूजचा न्यूयॉर्क शहरात नैसर्गिक कारणांमुळे मृत्यू झाला.', 'mar_Deva eng_Latn विंडो शॉपिंगच्या उद्देशाप्रमाणे खिडकीत प्रदर्शित केलेल्या वस्तू बघताना तुम्हाला कदाचि

In [34]:
sample_preds = [[60726, 59836, 2134, 5172, 4, 2]]  # example prediction ids
sample_labels = [[60726, 59836, 2134, 5172, 4, -100]]  # example label ids
compute_metrics_indictrans((sample_preds, sample_labels), tokenizer=en_indic_tokenizer, metric=metric)

{'bleu': 100.0, 'gen_len': 1.0}

In [35]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 379480
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 47435
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 47435
    })
})

In [36]:
# Example: pick 5 validation examples
sample_batch = tokenized_datasets["validation"][:5]
# print(sample_batch)
decode_tokenized_batch(en_indic_tokenizer, sample_batch)

['mar_Deva eng_Latn आता सुरुवात करू. मी आधीच माझ्या सिस्टिमवर Koha इन्स्टॉल केले आहे.', 'mar_Deva eng_Latn फक्त दोन खरोखरच्या दुहेरी कॉन्सर्टोपैकी एक तो आहे, जे दोन्ही मोत्झार्तने लिहिले.', 'mar_Deva eng_Latn थोडक्यात,', 'mar_Deva eng_Latn ह्या पुस्तकात आपण ह्या संकल्पनांचा वापर केला नाही पाहिजे आणि ज्यांना पुढे वाचण्याची इच्छा आहे, त्यांच्या उपयोगासाठी फक्त हा उल्लेख केला पाहिजे.', 'mar_Deva eng_Latn ती युक्रेनियन नॅशनल फिलहारमोनिकची एकटी वादक होती.']


['mar_Deva eng_Latn आता सुरुवात करू. मी आधीच माझ्या सिस्टिमवर Koha इन्स्टॉल केले आहे.',
 'mar_Deva eng_Latn फक्त दोन खरोखरच्या दुहेरी कॉन्सर्टोपैकी एक तो आहे, जे दोन्ही मोत्झार्तने लिहिले.',
 'mar_Deva eng_Latn थोडक्यात,',
 'mar_Deva eng_Latn ह्या पुस्तकात आपण ह्या संकल्पनांचा वापर केला नाही पाहिजे आणि ज्यांना पुढे वाचण्याची इच्छा आहे, त्यांच्या उपयोगासाठी फक्त हा उल्लेख केला पाहिजे.',
 'mar_Deva eng_Latn ती युक्रेनियन नॅशनल फिलहारमोनिकची एकटी वादक होती.']

In [37]:
import numpy as np

# Pick 5 examples from validation
sample_batch = tokenized_datasets["validation"].select(range(5))

# Use labels as "mock predictions" (or you can modify them slightly)
mock_preds = np.array(sample_batch["labels"])

# Prepare eval_preds tuple
# Normally: (predictions, labels)
eval_preds = (mock_preds, np.array(sample_batch["labels"]))

# Compute metrics
results = compute_metrics_indictrans(
    eval_preds,
    tokenizer=en_indic_tokenizer,
    metric=metric,
    source_lang="eng_Latn",
    target_lang="mar_Deva"
)

# Decode predictions and labels for inspection
decoded_preds = en_indic_tokenizer.batch_decode(
    mock_preds, skip_special_tokens=True, clean_up_tokenization_spaces=True
)
decoded_labels = en_indic_tokenizer.batch_decode(
    np.where(np.array(sample_batch["labels"]) != -100,
             np.array(sample_batch["labels"]),
             en_indic_tokenizer.pad_token_id),
    skip_special_tokens=True, clean_up_tokenization_spaces=True
)

# Print for visual check
print("Sample Mock Predictions vs References:\n")
for pred, ref in zip(decoded_preds, decoded_labels):
    print(f"Prediction: {pred}")
    print(f"Reference:  {ref}")
    print("-----")

print("Metrics:", results)

Sample Mock Predictions vs References:

Prediction: mar_Deva eng_Latn आता सुरुवात करू. मी आधीच माझ्या सिस्टिमवर Koha इन्स्टॉल केले आहे.
Reference:  mar_Deva eng_Latn आता सुरुवात करू. मी आधीच माझ्या सिस्टिमवर Koha इन्स्टॉल केले आहे.
-----
Prediction: mar_Deva eng_Latn फक्त दोन खरोखरच्या दुहेरी कॉन्सर्टोपैकी एक तो आहे, जे दोन्ही मोत्झार्तने लिहिले.
Reference:  mar_Deva eng_Latn फक्त दोन खरोखरच्या दुहेरी कॉन्सर्टोपैकी एक तो आहे, जे दोन्ही मोत्झार्तने लिहिले.
-----
Prediction: mar_Deva eng_Latn थोडक्यात,
Reference:  mar_Deva eng_Latn थोडक्यात,
-----
Prediction: mar_Deva eng_Latn ह्या पुस्तकात आपण ह्या संकल्पनांचा वापर केला नाही पाहिजे आणि ज्यांना पुढे वाचण्याची इच्छा आहे, त्यांच्या उपयोगासाठी फक्त हा उल्लेख केला पाहिजे.
Reference:  mar_Deva eng_Latn ह्या पुस्तकात आपण ह्या संकल्पनांचा वापर केला नाही पाहिजे आणि ज्यांना पुढे वाचण्याची इच्छा आहे, त्यांच्या उपयोगासाठी फक्त हा उल्लेख केला पाहिजे.
-----
Prediction: mar_Deva eng_Latn ती युक्रेनियन नॅशनल फिलहारमोनिकची एकटी वादक होती.
Reference:  ma

Then we just need to pass all of this along with our datasets to the Seq2SeqTrainer:

In [78]:
def compute_metrics_indictrans(
    eval_preds, 
    tokenizer, 
    metric_bleu,
    metric_chrf=None,
    metric_comet=None,
    metric_bleurt=None,
    source_lang="eng_Latn",
    target_lang="mar_Deva"
):
    preds = eval_preds.predictions
    labels = eval_preds.label_ids
    inputs = eval_preds.inputs    # <-- needed for COMET

    # Convert 
    preds = preds.cpu().numpy() if isinstance(preds, torch.Tensor) else preds
    labels = labels.cpu().numpy() if isinstance(labels, torch.Tensor) else labels
    inputs = inputs.cpu().numpy() if isinstance(inputs, torch.Tensor) else inputs

    # Decode all three
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(np.where(labels!=-100, labels, tokenizer.pad_token_id),
                                            skip_special_tokens=True)
    decoded_sources = tokenizer.batch_decode(inputs, skip_special_tokens=True)

    # IndicTrans cleanup
    decoded_preds, decoded_labels = postprocess_indictrans_text(
        decoded_preds, decoded_labels, source_lang, target_lang
    )

    # BLEU
    bleu = metric_bleu.compute(predictions=decoded_preds, references=decoded_labels)["score"]

    # ChrF++
    chrfpp = None
    if metric_chrf is not None:
        chrfpp = metric_chrf.compute(
            predictions=decoded_preds,
            references=decoded_labels,
            word_order=2
        )["score"]
    
    flat_refs = [r[0] if isinstance(r, list) else r for r in decoded_labels]
    # COMET
    comet = None
    if metric_comet is not None:
        comet = metric_comet.compute(
            sources=decoded_sources,
            predictions=decoded_preds,
            references=flat_refs
        )["mean_score"]

    # BLEURT
    bleurt = None
    if metric_bleurt is not None:
        bleurt_scores = metric_bleurt.compute(
            predictions=decoded_preds,
            references=decoded_labels
        )["scores"]
        bleurt = float(np.mean(bleurt_scores))

    # Length
    prediction_lens = [np.count_nonzero(p != tokenizer.pad_token_id) for p in preds]
    gen_len = float(np.mean(prediction_lens))

    return {
        "bleu": round(bleu, 4),
        "chrfpp": round(chrfpp, 4) if chrfpp else None,
        "comet": round(comet, 4) if comet else None,
        "bleurt": round(bleurt, 4) if bleurt else None,
        "gen_len": round(gen_len, 4),
    }

In [40]:
!pip install git+https://github.com/google-research/bleurt.git

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-jx2lf9km
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-jx2lf9km
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.4/620.4 MB 92.6 MB/s  0:00:06 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 94.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 94.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 92.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 96.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 90.4 MB/s  0:00:00
   ━━━

In [79]:
from evaluate import load

metric_bleu   = load("sacrebleu")
metric_chrf   = load("chrf")
metric_comet  = load("comet")

Fetching 5 files: 100%|█████████████████████| 5/5 [00:00<00:00, 3855.06it/s]
Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.5.6. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/2760a223ac957f30acfb18c8aa649b01cf1d75f2/checkpoints/model.ckpt`
Encoder model frozen.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [45]:
!pip uninstall tensorflow

Found existing installation: tensorflow 2.20.0
Uninstalling tensorflow-2.20.0:
  Would remove:
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.20.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [80]:
from transformers import Seq2SeqTrainer, DataCollatorForSeq2Seq

# Data collator for seq2seq
data_collator = DataCollatorForSeq2Seq(
    tokenizer=en_indic_tokenizer,
    model=en_indic_model,
    padding="longest"
)

# Seq2SeqTrainer setup (future-proof)
trainer = Seq2SeqTrainer(
    model=en_indic_model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"].select(range(50)),
    # tokenizer=en_indic_tokenizer,  # deprecated
    processing_class=en_indic_tokenizer,  # use instead of tokenizer
    data_collator=data_collator,
    compute_metrics=lambda eval_preds: compute_metrics_indictrans(
        eval_preds,
        tokenizer=en_indic_tokenizer,
        metric_bleu=metric_bleu,
        metric_chrf=metric_chrf,
        metric_comet=metric_comet
    )
)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [39]:
# trainer = Seq2SeqTrainer(
#     model,
#     args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["validation"],
#     data_collator=data_collator,
#     tokenizer=en_indic_tokenizer,
#     compute_metrics=compute_metrics
# )

We can now finetune our model by just calling the train method:

In [40]:
en_indic_model = en_indic_model.to(DEVICE)
# if en_indic_model == "cuda":
#     model.half()   # <-- this casts everything to FP16 manually

In [47]:
small_eval_dataset = tokenized_datasets["validation"].select(range(50))
print(small_eval_dataset["labels"][:5])

[[60726, 59836, 2134, 5172, 43144, 67156, 59836, 3233, 11007, 7130, 1139, 5590, 1711, 4, 125, 22815, 2353, 8980, 1496, 402, 62106, 16945, 7791, 5221, 975, 38, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [60726, 59836, 2134, 5172, 43144, 67156, 59836, 3233, 11007, 7130, 1597, 1764, 12003, 332, 45551, 4830, 3985, 1006, 53473, 11313, 24, 51, 38, 5, 169, 3166, 644, 183, 21560, 1394, 61, 27220, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [60726, 59836, 2134, 5172, 43144, 67156, 59836, 3233, 11007, 7130, 45924, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [60726, 59836, 2134, 5172, 43144, 67156, 59836, 3233, 11007, 7130, 3473, 50201, 520, 3473, 10526, 1860, 317, 2539, 1135, 135, 4424, 68, 40007, 4998, 4591, 5502, 2569, 38, 5, 2269, 232, 2545, 1597, 407, 2160, 1135, 4424, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [60726, 59836, 2134, 5172, 43144, 67156, 59836, 3233, 11007, 7130, 440, 11170, 1680

In [81]:
trainer.train()

Step,Training Loss,Validation Loss,Bleu,Chrfpp,Comet,Bleurt,Gen Len
50,No log,1.211616,6.519700,30.601400,0.535000,None,20.920000
100,0.930600,0.790579,7.039900,30.181100,0.522000,None,20.900000


💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Trainer is attempting to log a value of "None" of type <class 'NoneType'> for key "eval/bleurt" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Trainer is attempting to log a value of "None" of type <class 'NoneType'> for key "eval/bleurt" as a scalar. This invocation of Tensorboard's

KeyboardInterrupt: 

In [ ]:
trainer.push_to_hub()

## Test

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

checkpoint_dir = "thenlpresearcher/iitb_punct_robust_finetuned_eng_Ltn_to_mar_Deva"

# Load model (custom class loaded via trust_remote_code=True)
model = AutoModelForSeq2SeqLM.from_pretrained(
    checkpoint_dir,
    trust_remote_code=True
)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(DEVICE)

# Load tokenizer (custom tokenizer class loaded via trust_remote_code=True)
tokenizer = en_indic_tokenizer

In [ ]:
from IndicTransToolkit.processor import IndicProcessor

In [ ]:
import torch
from datasets import load_metric
from tqdm import tqdm
import pandas as pd


ip = IndicProcessor(inference=True)

BATCH_SIZE = 32 # Adjust for your GPU memory

src_lang, tgt_lang = "eng_Latn", "mar_Deva"

# Collect source and reference sentences
src_sentences = raw_datasets["test"]["src"][:1000]
ref_sentences = raw_datasets["test"]["tgt"][:1000]


prefix = f"{tgt_lang} {src_lang}"

def remove_prefix(text):
    if text.startswith(prefix):
        return text[len(prefix):].strip()
    return text.strip()

# Translate in batches
all_translations = []
for i in tqdm(range(0, len(src_sentences), BATCH_SIZE)):
    batch = src_sentences[i:i+BATCH_SIZE]
    translations = batch_translate(
        batch,
        src_lang,
        tgt_lang,
        model,
        tokenizer,
        ip,
        device=DEVICE
    )
    all_translations.extend(translations)

p_t = []
for t in all_translations:
    p_t.append(remove_prefix(t))

# Save translations alongside references
results_df = pd.DataFrame({
    "src": src_sentences,
    "reference": ref_sentences,
    "prediction": p_t
})
results_df.to_csv("indictrans2_robust_test_predictions_test.csv", index=False)

print("Translations saved to indictrans2_test_predictions.csv")

# ---- Evaluation metrics ---- #
bleu = load_metric("sacrebleu")
chrf = load_metric("chrf")

# Format for sacrebleu: references need to be list of lists
bleu_score = bleu.compute(predictions=all_translations, references=[[r] for r in ref_sentences])
chrf_score = chrf.compute(predictions=all_translations, references=[[r] for r in ref_sentences])

print(f"BLEU: {bleu_score['score']:.2f}")
print(f"chrF++: {chrf_score['score']:.2f}")

# Optional: save metrics
with open("indictrans2_test_metrics.txt", "w") as f:
    f.write(f"BLEU: {bleu_score['score']:.2f}\n")
    f.write(f"chrF++: {chrf_score['score']:.2f}\n")

print("Evaluation complete.")

In [ ]:
results_df.head()

In [88]:
results_df['prediction'][0]

'अफगाणिस्तानमध्ये हिंदू फारच कमी असले तरी मालदीवमध्ये हिंदू नाहीत.'

In [74]:
print(len(tokenizer))  # should match what you trained on

122706


In [50]:
import torch

def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip, device, batch_size=8, max_length=256):
    """
    Translate a batch of sentences using a seq2seq model like IndicTrans with safety checks.

    Args:
        input_sentences (list of str): Source sentences to translate.
        src_lang (str): Source language code, e.g., "eng_Latn".
        tgt_lang (str): Target language code, e.g., "mar_Deva".
        model: Hugging Face seq2seq model.
        tokenizer: Corresponding tokenizer.
        ip: Preprocessing object (IndicProcessor).
        device: torch device ("cuda" or "cpu").
        batch_size (int): Batch size for generation.
        max_length (int): Maximum length of generated sequence.

    Returns:
        translations (list of str): Translated sentences.
    """
    model.to(device)
    model.eval()
    translations = []

    # Safe access for decoder_start_token_id
    decoder_start_token_id = getattr(model.config, "decoder_start_token_id", None)
    pad_token_id = getattr(tokenizer, "pad_token_id", None)
    eos_token_id = getattr(tokenizer, "eos_token_id", None)

    if decoder_start_token_id is None:
        print("[Warning] decoder_start_token_id is None. Using default generation behavior.")

    for i in range(0, len(input_sentences), batch_size):
        batch = input_sentences[i : i + batch_size]

        # Preprocess the batch
        batch_preprocessed = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)
        if not isinstance(batch_preprocessed, list) or len(batch_preprocessed) == 0:
            print(f"[Warning] Preprocessed batch is empty at index {i}. Skipping...")
            continue

        # Debug: print first 2 sentences after preprocessing
#         print(f"[Debug] Preprocessed batch sample: {batch_preprocessed[:2]}")

        # Tokenize
        inputs = tokenizer(
            batch_preprocessed,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        )

        # Move tensors to the correct device
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Generate translations with safety parameters
        with torch.no_grad():
            try:
                generated_tokens = model.generate(
                    **inputs,
                    use_cache=True,
                    min_length=5,
                    max_length=max_length,
                    num_beams=5,
                    num_return_sequences=1,
                    early_stopping=True,
                    decoder_start_token_id=decoder_start_token_id,
                    pad_token_id=pad_token_id,
                    eos_token_id=eos_token_id
                )
            except Exception as e:
                print(f"[Error] Generation failed for batch starting at index {i}: {e}")
                continue

        # Decode generated tokens
        decoded_texts = tokenizer.batch_decode(
            generated_tokens,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )

        # Debug: print first 2 decoded outputs
#         print(f"[Debug] Decoded sample: {decoded_texts[:2]}")

        # Postprocess translations
        try:
            postprocessed = ip.postprocess_batch(decoded_texts, lang=tgt_lang)
            translations += postprocessed
        except Exception as e:
            print(f"[Error] Postprocessing failed for batch starting at index {i}: {e}")
            translations += decoded_texts  # fallback

        # Free GPU memory
        del inputs, generated_tokens
        torch.cuda.empty_cache()

    return translations